# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [2]:
import re
import requests
from bs4 import BeautifulSoup


### 先搜尋全部的電影代號(ID)資訊

In [3]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
r=requests.get(url)
r.encoding='utf-8'

soup=BeautifulSoup(r.text,'lxml')
print(soup.prettify())

AE%B6-matriarch-10515">
             電影介紹
            </a>
            <a class="btn_s_vedio gabtn" data-ga="['上映中','上映中_上映中第1頁','囚愛之家']" href="https://movies.yahoo.com.tw/video/%E5%9B%9A%E6%84%9B%E4%B9%8B%E5%AE%B6-%E4%B8%AD%E6%96%87%E9%A0%90%E5%91%8A-033841822.html?movie_id=10515">
             預告片
            </a>
            <a class="btn_s_foto" href="https://movies.yahoo.com.tw/movieinfo_photos.html/id=10515">
             劇照
            </a>
            <a class="btn_s_time gabtn" data-ga="['上映中','上映中_上映中第1頁','囚愛之家']" href="https://movies.yahoo.com.tw/movietime_result.html/id=10515">
             時刻表
            </a>
           </div>
          </div>
         </li>
         <li>
          <div class="release_foto">
           <a class="gabtn" data-ga="['上映中','上映中_上映中第1頁','神之一手：鬼手篇']" href="https://movies.yahoo.com.tw/movieinfo_main/%E7%A5%9E%E4%B9%8B%E4%B8%80%E6%89%8B-%E9%AC%BC%E6%89%8B%E7%AF%87-the-divine-move-2-the-wrathful-10520">
            <img alt="" src="https://movies.y

In [4]:
movie_url=soup.find_all('a',class_="btn_s_vedio gabtn")
movie_id=[]
movie=[]
for link in movie_url:
    movie_id.append(link.get('href').split("movie_id=")[1])
movie_name_old=soup.find_all("div",class_="release_movie_name")
movie_name=[item.a.text.replace(" ","").replace("\n","") for item in movie_name_old]
for i in range(0,len(movie_id)):
    movie.append({"id":movie_id[i],"name":movie_name[i]})
print(movie)

[{'id': '10487', 'name': '依然相信'}, {'id': '10494', 'name': '我出去醫下'}, {'id': '10499', 'name': '最後晚安曲'}, {'id': '10502', 'name': '搖滾師匠：細野晴臣'}, {'id': '10508', 'name': 'A+瞎妹'}, {'id': '10515', 'name': '囚愛之家'}, {'id': '10520', 'name': '神之一手：鬼手篇'}, {'id': '10542', 'name': '不愛鋼琴師'}, {'id': '10546', 'name': '我的失蹤爸爸'}, {'id': '10432', 'name': '鋼鐵勳章'}]


### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [5]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
s_movieid=soup.find("select",attrs={"name":"movie_id"}).find_all("option",attrs={'data-name':re.compile('.*')})
# for item in s_movieid:
#     print(item.get("value"))
s_movie=[{"name":item.text,"id":item.get("value")}for item in s_movieid]
print(s_movie)

[{'name': '我的失蹤爸爸', 'id': '10546'}, {'name': '不愛鋼琴師', 'id': '10542'}, {'name': '神之一手：鬼手篇', 'id': '10520'}, {'name': '囚愛之家', 'id': '10515'}, {'name': 'A+瞎妹', 'id': '10508'}, {'name': '搖滾師匠：細野晴臣', 'id': '10502'}, {'name': '最後晚安曲', 'id': '10499'}, {'name': '我出去醫下', 'id': '10494'}, {'name': '依然相信', 'id': '10487'}, {'name': '電影版 地獄少女', 'id': '10525'}, {'name': '刺激1995', 'id': '10518'}, {'name': '通靈少年2020', 'id': '10517'}, {'name': '安雅的回家路', 'id': '10516'}, {'name': '牆內的房客', 'id': '10512'}, {'name': '南山的部長們', 'id': '10509'}, {'name': '另類神父', 'id': '10500'}, {'name': '破．局', 'id': '10496'}, {'name': '劇場版 不起眼女主角培育法 Fine', 'id': '10486'}, {'name': '複製柔伊', 'id': '10455'}, {'name': '練愛iNG', 'id': '10442'}, {'name': '鋼鐵勳章', 'id': '10432'}, {'name': '我，最親愛的', 'id': '10503'}, {'name': '血衛', 'id': '10469'}, {'name': '導演先生的完美假期', 'id': '10545'}, {'name': '大地蜜語', 'id': '10513'}, {'name': '狂飆一夢', 'id': '10506'}, {'name': '親愛的莎瑪', 'id': '10491'}, {'name': '大畫特務', 'id': '10478'}, {'name': '戰慄糖果屋', 'id': '1

In [6]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
# f12 中丟入值會得到的當案url 會回傳 response整個html的資料
idmovie=s_movie[5]['id']
# print(idmovie)
#檔案下面有一個 Query  String Parameters裡面有寫要甚麼，放進payload裡面
payload = {'movie_id':idmovie}
area_id = 28
# 模擬一個header
#不需要這麼多東西，但第一次可以先全部加進來。
headers = {
    'path': '/movietime_result.html?movie_id='+idmovie+'&area_id=28',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
}
    
resp = requests.get(url, params=payload, headers=headers)
# print(resp.json())  # 若有需要，列印出json原始碼
# 接js回傳而改變html的地方


# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 桃園, 代號(area_id): 16
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 屏東, 代號(area_id): 14


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [7]:
# 指定放映地區


In [8]:
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(idmovie), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

三月 23
三月 24
三月 25
三月 26
三月 27


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [33]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''
# print(soup.prettify())
name=soup.find_all('h1',class_="inform_title")
name_list=[{"name":item.text}for item in name]
print(name_list[0])

{'name': '搖滾師匠：細野晴臣\n      No Smoking\n'}
